## Tworzenie ramki danych do trenowania modelu

In [ ]:
from typing import Dict, List, Any, Self, Optional

import numpy as np
# Konkatenacja powstałych datasetów csv w 1, wraz z transformacją kolumn math, bio, code w jedną - category,
# której jedyne wartości 0, 1, 2 odnoszą się odpowiednio do kategorii math, bio, code.

import pandas as pd
import glob
import sys
import os

from matplotlib.pyplot import title

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

csv_files = glob.glob('../datasets_preprocessing/csv_question_files/*.csv')

dfs = []
for file in csv_files:
    tmpdf = pd.read_csv(file)
    sample = tmpdf.sample(n=5000)
    dfs.append(sample)

df = pd.concat(dfs, ignore_index=True)

def transform_categories(math, bio, code):
    if math == 1:
        return 0
    elif bio == 1:
        return 1
    elif code == 1:
        return 2
    return 2

df['category'] = df.apply(lambda row: transform_categories(row['math'], row['bio'], row['code']), axis=1)

# Usunięcie kolumn math, bio, code z df — są zastąpione category
df.drop(['math', 'bio', 'code'], axis=1, inplace=True)

In [ ]:
df.sample(n=10)

## Model SVM z vectorizer'em TF-IDF

In [ ]:
from train.reporting.model_reporter import ModelReporter
from train.reporting.svm_model_wrapper import SVMModelWrapper
from sklearn.feature_extraction.text import TfidfVectorizer

model = SVMModelWrapper('tfidf', TfidfVectorizer(stop_words='english'), C=1.0)
reporter = ModelReporter(model, df['question'], df['category'])

reporter.generate_report()

## Generowanie wykresu TSNE do określenia działania vectorizer'a

In [ ]:
reporter.plot_tsne()

## Generowanie wykresów wordclouds dla każdej kategorii

In [ ]:
reporter.plot_wordclouds()

## Generowanie wykresu rozkładu pewności modelu

In [ ]:
reporter.plot_confidence_distribution()